In [1]:
%load_ext autoreload
%autoreload 2


In [3]:
from src.processors.Preprocessor import Preprocessor
from src.model_experiemtation.ModelExperimentor import TFModelExperimentor

import datetime
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
from sklearn.model_selection import train_test_split
import tensorflow as tf


import tensorflow as tf
from tensorflow.keras.layers import Dense, LayerNormalization, LSTM
from tensorflow.keras import Sequential

from tensorflow.keras.losses import Huber, MeanSquaredError

#validate GPU detection
print("TensorFlow version:", tf.__version__)
for gpu in tf.config.list_physical_devices('GPU'):
    print(gpu)



2023-05-26 13:56:30.685590: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-26 13:56:33.721489: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 13:56:43.135441: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.12.0


In [4]:
PATH = '/home/mds8301/gaby_test/processors/unit_test_processor.pkl'
proc = Preprocessor().load_processor(PATH)
proc.one_hot_encode(labels = ['event', 'sensor'])

: 

: 

## initial prototyping plan
* query trials below 20 for training
* query trials above 20 for testing and prediction
* drop mouse_id 




In [ ]:
#downsample data to reduce training timne
downsample_data = proc.processed_data.drop(proc.processed_data.index[::100])
downsample_data.to_parquet('/home/mds8301/gaby_test/processors/downsampled_data.parquet')


memory reducetion


In [ ]:
downsample_data.memory_usage(deep=True).sum()/proc.processed_data.memory_usage(deep=True).sum()

In [ ]:
plt.plot(downsample_data.query('mouse_id == "310_909" & trial==7 & sensor_DA == 1 & event_avoid==1')['time'], downsample_data.query('mouse_id == "310_909" & trial==7 & sensor_DA == 1 & event_avoid==1')['signal'])
plt.show()

### split into training and test sets by trial

> query data for a specific mouse, event avoid and dopamine sensor to further reduce training time

In [ ]:
query = 'mouse_id == "310_909" & sensor_DA == 1 & event_avoid==1'
non_useful_columns=['mouse_id','event_cue','event_escape', 'event_shock','sensor_D1', 'sensor_D2']
mouse_909_DA_avoid = downsample_data.query(query).drop(columns=non_useful_columns)
mouse_909_DA_avoid

In [ ]:

trials_under_5 = mouse_909_DA_avoid.query('trial<=5')
trials_over_5 = mouse_909_DA_avoid.query('trial>5')

X_train, y_train = trials_under_5.drop(columns = ['signal']), trials_under_5.signal
X_test, y_test = trials_over_5.drop(columns = ['signal']), trials_over_5.signal

In [ ]:


sequential_model = Sequential(
    [
        Dense(128, activation='relu', name="dense_1"),
        Dense(1, activation='relu', name="dense_2")
    ]
)


In [ ]:
model_id = 'sequential_prototype'
date_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
training_log_dir = "/projects/p31961/dopamine_modeling/results/logs/training_logs/" 
logs_dir = f"{training_log_dir}/{model_id}/{date_time}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_dir, histogram_freq=1)

In [ ]:
sequential_model.compile(
    optimizer="adam", loss=Huber())
sequential_model.fit(X_train, y_train, batch_size=30, epochs= 100, callbacks=[tensorboard_callback])
prediction = sequential_model.predict(X_test)
sequential_model.evaluate(X_test, y_test)

In [ ]:
# save model
model_dir = "/projects/p31961/dopamine_modeling/results/models/"
sequential_model.save(f"{model_dir}/{model_id}/{date_time}")
